<a href="https://colab.research.google.com/github/kridtapon/Cloud-Optimized-Momentum/blob/main/Cloud_Optimized_Momentum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install vectorbt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.6/527.6 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 36.8 MB/s eta 0:00:00


In [10]:
import numpy as np
import pandas as pd
import yfinance as yf
import vectorbt as vbt

# Function to calculate Ichimoku Cloud components
def calculate_ichimoku(df, tenkan_period=9, kijun_period=26, senkou_span_b_period=52):
    """
    Calculate Ichimoku Cloud indicators.
    """
    # Tenkan-sen (Conversion Line) = (Highest High + Lowest Low) / 2 over the last 9 periods
    high_9 = df['High'].rolling(window=tenkan_period).max()
    low_9 = df['Low'].rolling(window=tenkan_period).min()
    df['Tenkan-sen'] = (high_9 + low_9) / 2

    # Kijun-sen (Base Line) = (Highest High + Lowest Low) / 2 over the last 26 periods
    high_26 = df['High'].rolling(window=kijun_period).max()
    low_26 = df['Low'].rolling(window=kijun_period).min()
    df['Kijun-sen'] = (high_26 + low_26) / 2

    # Senkou Span A = (Tenkan-sen + Kijun-sen) / 2, plotted 26 periods ahead
    df['Senkou Span A'] = ((df['Tenkan-sen'] + df['Kijun-sen']) / 2).shift(kijun_period)

    # Senkou Span B = (Highest High + Lowest Low) / 2 over the last 52 periods, plotted 26 periods ahead
    high_52 = df['High'].rolling(window=senkou_span_b_period).max()
    low_52 = df['Low'].rolling(window=senkou_span_b_period).min()
    df['Senkou Span B'] = ((high_52 + low_52) / 2).shift(kijun_period)

    # Chikou Span = Closing Price plotted 26 periods back
    df['Chikou Span'] = df['Close'].shift(-kijun_period)

    return df

# Define the stock symbol and time period
symbol = 'AXON'  # SPY is the symbol for the S&P 500 ETF
start_date = '2019-01-01'
end_date = '2025-01-01'

# Download the stock data
df = yf.download(symbol, start=start_date, end=end_date)
df.columns = ['Close', 'High', 'Low', 'Open', 'Volume']
df.ffill(inplace=True)

# Calculate Ichimoku Cloud components
df = calculate_ichimoku(df)

# Define Entry and Exit signals based on Ichimoku Cloud
# Entry: Price above the Cloud, Tenkan-sen above Kijun-sen
df['Entry'] = (df['Close'] > df['Senkou Span A']) & (df['Close'] > df['Senkou Span B']) & (df['Tenkan-sen'] > df['Kijun-sen'])

# Exit: Price below the Cloud or Tenkan-sen below Kijun-sen
df['Exit'] = (df['Close'] < df['Senkou Span A']) & (df['Close'] < df['Senkou Span B']) & (df['Tenkan-sen'] < df['Kijun-sen'])

# Filter data for the test period (2020-2025)
df = df[(df.index.year >= 2020) & (df.index.year <= 2025)]

# Backtest using vectorbt
portfolio = vbt.Portfolio.from_signals(
    close=df['Close'],
    entries=df['Entry'],
    exits=df['Exit'],
    init_cash=100_000,
    fees=0.001
)

# Display performance metrics
print(portfolio.stats())

# Plot equity curve
portfolio.plot().show()


[*********************100%***********************]  1 of 1 completed
/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sharpe_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'calmar_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'omega_ratio' requires frequency to be set

/usr/local/lib/python3.11/dist-packages/vectorbt/generic/stats_builder.py:396: UserWarning:

Metric 'sortino_ratio' requires frequency to be set



Start                         2020-01-02 00:00:00
End                           2024-12-31 00:00:00
Period                                       1258
Start Value                              100000.0
End Value                           417991.564007
Total Return [%]                       317.991564
Benchmark Return [%]                   676.280076
Max Gross Exposure [%]                      100.0
Total Fees Paid                       3650.948783
Max Drawdown [%]                        40.467162
Max Drawdown Duration                       443.0
Total Trades                                   11
Total Closed Trades                            10
Total Open Trades                               1
Open Trade PnL                      154908.602108
Win Rate [%]                                 50.0
Best Trade [%]                           79.11981
Worst Trade [%]                        -12.808806
Avg Winning Trade [%]                   37.000019
Avg Losing Trade [%]                    -9.476361
